# More on Bayesian Neural Networks

As we have seen in this course training a full bayesian neural network is on state-of-the-art and several problems exist: slow convergence, high variance, too simple posteriors, etc

Training a Bayesian neural network using VI resorts to maximizing 

$$
\mathcal{L}(\nu) = \mathbb{E}_{\theta \sim q_\nu(\theta)} \left[\log p(\mathcal{D}|\theta)\right]- D_{KL}[q_\nu(\theta) || p(\theta)]
$$

where $\nu$ are the parameters of the approximate posterior

## Bayes by backprop

In 2015 [Blundel et al](https://arxiv.org/pdf/1505.05424.pdf) proposed **Bayes by Backprop** which consists on replacing the expectation above with monte-carlo estimates 

$$
\mathcal{L}(\nu) \approx  \sum_{i=1}^N \sum_{k=1}^K \log p(x_i|\theta_k)-  \log q_\nu(\theta_k)  + \log p(\theta_k)
$$

where $N$ is the number of data samples in the minibatch and $K$ is the number of times we sample from the parameters $\theta$. This formulation is more general (it does not depend on closed-form solutions of the KL)

Thanks to this flexibility more complex priors can be used. In the original Bayes-by-backprop paper the following is considered

$$
p(\theta) = \pi_1 \mathcal{N}(0, \sigma_1^2) + \pi_2 \mathcal{N}(0, \sigma_2^2)
$$

with $\sigma_1<<<\sigma_2$. The term with smaller variance allows for automatic "shut-down" (pruning) of weights, i.e. sparsification. Note that [Gaussian scale mixtures are implemented in Pyro](http://docs.pyro.ai/en/stable/distributions.html#pyro.distributions.GaussianScaleMixture) but are a bit tricky to use

Other implementations of Bayes by backprop

- https://www.nitarshan.com/bayes-by-backprop/
- http://krasserm.github.io/2019/03/14/bayesian-neural-networks/
- https://gluon.mxnet.io/chapter18_variational-methods-and-uncertainty/bayes-by-backprop.html

## [Local reparametrization trick to reduce noise](http://papers.nips.cc/paper/5666-variational-dropout-and-the-local-reparameterization-trick)

In BNN we sample from every weight as 

$$
w_{ji}\sim \mathcal{N}(\mu_{ji}, \sigma_{ji}^2)
$$

using the reperameterization trick


$$
w_{ji} = \mu_{ji} +\epsilon_{ji} \cdot\sigma_{ji}, \quad \epsilon_{ji} \sim \mathcal{N}(0, I)
$$

but we can reduce the variance further. The idea behind local reparameterization is that instead of sampling from every weight we sample from the pre-activations

$$
Z = WX + B
$$

then

$$
z_i = \nu_i + \eta_i  \cdot \epsilon_{i}
$$

where $\epsilon$ is still a standard normal and $\nu_i = \sum_j x_j \mu_{ji}$ and $\eta_i = \sqrt{\sum_j x_j^2 \sigma_{ji}^2}$

This reduces the amounts of samples we take by orders and magnitude and also reduces the variance of the estimator. The local reparameterization is [implementation in `pyro`](http://docs.pyro.ai/en/stable/contrib.bnn.html#pyro.contrib.bnn.hidden_layer.HiddenLayer) and a demonstration can be found [here](https://alsibahi.xyz/snippets/2019/06/15/pyro_mnist_bnn_kl.html)

## [Dropout as a Bayesian approximation](https://arxiv.org/abs/1506.02142)

This is an alternative take on Bayesian neural networks based on the dropout technique for regularization

Dropout turns-off neurons following a certain distribution. The authors argue that this is like having an ensemble of neural networks and hence uncertainties can be computed. This is done by applying dropout not only during training but also when predicting (test set) to estimate uncertainty

[This short letter](http://bayesiandeeplearning.org/2016/papers/BDL_4.pdf) critiques this application of dropout, and shows that uncertainty with this approach (fixed dropout probability) does not decrease as new data points arrive. [A solution to this?](https://papers.nips.cc/paper/6949-concrete-dropout)


## [FLIPOUT](https://arxiv.org/abs/1803.04386)

Decorrelation of the gradients within a minibatch speeding up bayesian neural networks with gaussian perturbations

## [Natural gradient VI](https://papers.nips.cc/paper/8681-practical-deep-learning-with-bayesian-principles.pdf)

# Advances in MCMC

If MCMC would be faster we probably would not need VI

- https://github.com/pyro-ppl/numpyro
- [Approximate MCMC](https://arxiv.org/abs/1908.03491)
